In [1]:
import pandas as pd
import altair as alt
from sklearn.feature_extraction.text import TfidfVectorizer
alt.renderers.enable('notebook')
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

In [2]:
df = pd.read_csv('humanist_vols_ner_tokens.csv')

In [4]:
# Let's test out TF-IDF vs Word Counts vs NER

In [5]:
# TFIDF CODE

In [ ]:
all_docs = df.cleaned_text.tolist()

# Specify the ngram_range for the Vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.5, min_df=1)
transformed_documents = vectorizer.fit_transform(all_docs)

transformed_documents_as_array = transformed_documents.toarray()

# Output the top tokens for each document
for counter, doc in enumerate(transformed_documents_as_array):
    # construct a dataframe
    tf_idf_tuples = list(zip(vectorizer.get_feature_names(), doc))
    one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)
    print(one_doc_as_df[0:20])

In [6]:
# Word Counts

In [ ]:
df_exploded = df.explode('cleaned_tokens')
df_grouped = df_exploded.groupby(['dates', 'cleaned_tokens']).size().reset_index(name='frequency')
df_grouped.sort_values(by=['dates', 'frequency' ], inplace=True, ascending=False)
df_top = df_grouped.sort_values(by=['dates','frequency'],ascending = False).groupby('dates').head(10)
df_pivot = df_top.pivot(index='cleaned_tokens', columns='dates', values='frequency').fillna(0)
df_top = df_pivot.unstack().reset_index(name='frequency')

In [ ]:
alt.Chart(df_top).mark_area(point=True).encode(
    y='frequency:Q',
    x='dates',
    color='cleaned_tokens:N',
    tooltip=['cleaned_tokens', 'frequency', 'dates'],
    row='cleaned_tokens:N'
).properties(height=50)

In [ ]:
# NER Counts

In [ ]:
df['ner_tokens']= df.ner.apply(nltk.word_tokenize)
df['ner_counts'] = df.ner_tokens.str.len()
df['year'] = pd.to_datetime(df.dates.str[0:4], format='%Y')
alt.Chart(df[['year', 'ner_counts']]).mark_bar().encode(
    x='year:T',
    y='ner_counts:Q'
)

In [ ]:
alt.Chart(df[['original_text_length', 'ner_counts', 'year']]).mark_point().encode(
    x='original_text_length:Q',
    y='ner_counts:Q',
    color='year:T'
)

In [ ]:
# nearest = alt.selection(type='single', nearest=True, on='mouseover',
#                         fields=['year'], empty='none')
# line = alt.Chart(df_top).mark_line().encode(
#     y='frequency:Q',
#     x='year:T',
#     color=alt.Color('ner_tokens:N', scale=alt.Scale(scheme='category20b')),
# )
# selectors = alt.Chart(df_top).mark_point().encode(
#     x='year:T',
#     opacity=alt.value(0),
# ).add_selection(
#     nearest
# )
# points = line.mark_point().encode(
#     opacity=alt.condition(nearest, alt.value(1), alt.value(0))
# )
# rules = alt.Chart(df_top).mark_rule(color='gray').encode(
#     x='year:T',
# ).transform_filter(
#     nearest
# )
# text = line.mark_text(align='left', dx=5, dy=-5).encode(
#     text=alt.condition(nearest, 'frequency:Q', alt.value(' '))
# )

# alt.layer(
#     line, selectors, points, rules, text
# ).properties(
#     width=600, height=500
# )